In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import numpy as np
import json
import numpy as np
import argparse
import tempfile
import time
import tarfile
import sagemaker
import boto3

from pathlib import Path
from botocore.exceptions import ClientError
from sagemaker.workflow.model_step import ModelStep
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import CacheConfig
from sagemaker.tensorflow import TensorFlow
from sagemaker.workflow.steps import TrainingStep
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker import ModelPackage
from sagemaker.tensorflow import TensorFlowProcessor
from sagemaker.model import Model
from sagemaker.tensorflow.model import TensorFlowModel
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.functions import JsonGet
from sagemaker.workflow.functions import Join
from sagemaker.workflow.properties import PropertyFile
from sagemaker.tensorflow.model import TensorFlowModel
from sagemaker.tensorflow.model import TensorFlowPredictor


In [4]:
role = sagemaker.get_execution_role()
sagemaker_client = boto3.client("sagemaker")
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()

In [5]:
MODEL_S3_URI = "s3://sagemaker-us-east-1-325223348818/pipelines-9cal8byufuxp-train-model-HsrPe42D9A/output/model.tar.gz"
LOCAL_FOLDER = "sample"

Path(LOCAL_FOLDER).mkdir(parents=True, exist_ok=True)

In [6]:
%%writefile {LOCAL_FOLDER}/inference.py

import os
import json
import boto3
import requests

from pathlib import Path


def handler(data, context):
    """
    This is the entrypoint that will be called by SageMaker when the endpoint
    receives a request. You can see more information at 
    https://github.com/aws/sagemaker-tensorflow-serving-container.
    """
    print("Handling endpoint request")
    
    data = _process_input(data, context)
    output = _predict(data, context)
    return _process_output(output, context)


def _process_input(data, context):
    return data


def _predict(instance, context):
    print("Sending input data to model to make a prediction...")
    
    model_input = json.dumps({"instances": [instance]})
    
    response = requests.post(context.rest_uri, data=model_input)

    if response.status_code != 200:
        raise ValueError(response.content.decode('utf-8'))

    result = json.loads(response.content)
    
    print(f"Response: {result}")
    return result


def _process_output(output, context):
    print("Processing prediction received from the model...")
    
    response_content_type = "application/json" if context is None else context.accept_header
    
    prediction = np.argmax(output["predictions"][0])
    confidence = output["predictions"][0][prediction]
    
    print(f"Prediction: {prediction}. Confidence: {confidence}")
    
    result = json.dumps({
        "prediction": int(prediction),
        "confidence": confidence
    }), response_content_type
    
    return result


Overwriting sample/inference.py


In [7]:
%%writefile {LOCAL_FOLDER}/requirements.txt

numpy==1.19.5
pandas==1.2.5
scikit-learn==0.23.2

Overwriting sample/requirements.txt


In [8]:
model = TensorFlowModel(
    model_data=MODEL_S3_URI,
    entry_point="inference.py",
    source_dir=LOCAL_FOLDER,
    framework_version="2.6",
    sagemaker_session=PipelineSession(),
    role=role,
)

register_model_step = ModelStep(
    name="register",
    step_args=model.register(
        model_package_group_name="sample",
        approval_status="Approved",
        
        content_types=["application/json"],
        response_types=["application/json"],
        inference_instances=["ml.m5.large"],
        domain="MACHINE_LEARNING",
        task="CLASSIFICATION",
        framework="TENSORFLOW",
        framework_version="2.6",
    )
)

/usr/local/lib/python3.8/site-packages/sagemaker/workflow/pipeline_context.py:286: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


In [9]:
pipeline = Pipeline(
    name="model-registry-issue",
    steps=[
        register_model_step
    ],
)

pipeline.upsert(role_arn=role)
execution = pipeline.start()

Popping out 'CertifyForMarketplace' from the pipeline definition since it will be overridden in pipeline execution time.
